# Scoring and Citations Testbed
---

## Import Libraries

In [2]:
import sys 
import os 
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [7]:
import torch as t
import numpy as np
import pandas as pd
import torch.nn.functional as F
from tqdm.auto import tqdm

from transformers import GPTNeoXForCausalLM, AutoTokenizer
from datasets import load_dataset, Dataset
from sentence_transformers import SentenceTransformer, util

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from nltk.tokenize import sent_tokenize

from sentence_transformers import SentenceTransformer
from rouge_score import rouge_scorer
from sklearn.feature_extraction.text import TfidfVectorizer
import ast
import re
from scipy.spatial.distance import cosine


In [8]:
from src.model import load_fo_model, load_ba_model, DEVICE
from src.data import load_cnn_dataset
from src.utils import *
from src.search import linear_attribution_search

Using device: cuda


In [9]:
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.float_format', '{:.4f}'.format)

In [18]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /jet/home/iwiryadi/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [10]:
DEVICE

'cuda'

## Quick Test

In [11]:
# Load the models
fo_model, fo_tokenizer = load_fo_model()
ba_model, ba_tokenizer = load_ba_model()

# Example Text
article = "Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him."
summary = "Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday"
adverse_summary = "Daniel Craig is recasted as James Bond again"

/jet/home/iwiryadi/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/598 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/375M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/146 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
debug = True

# In normal, query is sentence/article, and answer is summary/highlight (S->A direction)
base = calculate_score(summary, article, fo_model, fo_tokenizer, backward=False, query_direction="normal", debug=debug)

# In Fo, query is summary/highlight , and answer is sentence/article(A->S direction)
fo = calculate_score(summary, article, fo_model, fo_tokenizer, backward=False, query_direction="reverse", debug=debug)

# In Ba, query is summary/highlight , and answer is sentence/article(A->S direction)
ba = calculate_score(summary, article, ba_model, ba_tokenizer, backward=True, query_direction="reverse", debug=debug)


adv_base = calculate_score(adverse_summary, article, fo_model, fo_tokenizer, backward=False, query_direction="normal", debug=debug)

adv_fo = calculate_score(adverse_summary, article, fo_model, fo_tokenizer, backward=False, query_direction="reverse", debug=debug)

adv_ba = calculate_score(adverse_summary, article, ba_model, ba_tokenizer, backward=True, query_direction="reverse", debug=debug)


scores_data = {
    'Model Type': ['Base', 'Forward', 'Backward', 'Adv Base', 'Adv Forward', 'Adv Backward'],
    
    'Query Direction': ['S->A', 'A->S', 'A->S', 'S->A', 'A->S', 'A->S'],
    
    'Sequence Log Prob': [
        base['sequence_log_prob'],
        fo['sequence_log_prob'],
        ba['sequence_log_prob'],
        
        adv_base['sequence_log_prob'],
        adv_fo['sequence_log_prob'],
        adv_ba['sequence_log_prob'],
    ],
    'Normalized Log Prob': [
        base['normalized_log_prob'],
        fo['normalized_log_prob'],
        ba['normalized_log_prob'],
        
        adv_base['normalized_log_prob'],
        adv_fo['normalized_log_prob'],
        adv_ba['normalized_log_prob'],
    ],
    'Perplexity': [
        base['perplexity'],
        fo['perplexity'],
        ba['perplexity'],
        
        adv_base['perplexity'],
        adv_fo['perplexity'],
        adv_ba['perplexity'],
    ]
}

Context: Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Mondayis a summary of
Target: Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him.
Full sentence: Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Mondayis a summary ofHarry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him.

Context: Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him.is a summary of
Target: Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday
Full sentence: Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists

In [13]:
result_df = pd.DataFrame(scores_data)
result_df

,Model Type,Query Direction,Sequence Log Prob,Normalized Log Prob,Perplexity
0,Base,S->A,-113.7697,-2.7749,16.0366
1,Forward,A->S,-48.3545,-3.0222,20.5355
2,Backward,A->S,-40.4211,-2.5263,12.5074
3,Adv Base,S->A,-156.6189,-3.8200,45.6030
4,Adv Forward,A->S,-65.3957,-7.2662,1431.0798
5,Adv Backward,A->S,-57.9534,-6.4393,625.9468


## Linear Search

In [20]:
dataset = load_cnn_dataset(num_samples=500)
dataset = pd.DataFrame(dataset)

Dataset loaded successfully
Example dataset item: {'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places be

In [21]:
dataset.head(5)

,article,highlights,id
0,"LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell...",Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday .\nYoung actor says he has no plans to fritter his cash away .\nRadcliffe's earnings from first five Potter films have be...,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4
1,"Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a ja...","Mentally ill inmates in Miami are housed on the ""forgotten floor""\nJudge Steven Leifman says most are there as a result of ""avoidable felonies""\nWhile CNN tours facility, patient shouts: ""I am the...",ee8871b15c50d0db17b0179a6d2beab35065f1e9
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who were on the Minneapolis bridge when it collapsed told harrowing tales of survival. ""The whole bridge from one side of the Mississippi to the other just ...","NEW: ""I thought I was going to die,"" driver says .\nMan says pickup truck was folded in half; he just has cut on face .\nDriver: ""I probably had a 30-, 35-foot free fall""\nMinnesota bridge collaps...",06352019a19ae31e527f37f7571c6dd7f0c5da37
3,"WASHINGTON (CNN) -- Doctors removed five small polyps from President Bush's colon on Saturday, and ""none appeared worrisome,"" a White House spokesman said. The polyps were removed and sent to the ...","Five small polyps found during procedure; ""none worrisome,"" spokesman says .\nPresident reclaims powers transferred to vice president .\nBush undergoes routine colonoscopy at Camp David .",24521a2abb2e1f5e34e6824e0f9e56904a2b0e88
4,"(CNN) -- The National Football League has indefinitely suspended Atlanta Falcons quarterback Michael Vick without pay, officials with the league said Friday. NFL star Michael Vick is set to appea...","NEW: NFL chief, Atlanta Falcons owner critical of Michael Vick's conduct .\nNFL suspends Falcons quarterback indefinitely without pay .\nVick admits funding dogfighting operation but says he did n...",7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a


In [22]:
linear_results = linear_attribution_search(dataset, fo_model, fo_tokenizer, ba_model, ba_tokenizer)

  0%|          | 0/500 [00:00<?, ?it/s]

In [23]:
def calculate_embedding_similarity(highlight, citation):
    """Calculate cosine similarity between sentence embeddings."""
    model = SentenceTransformer('all-MiniLM-L6-v2')
    
    # Generate embeddings
    highlight_embedding = model.encode([highlight])[0]
    citation_embedding = model.encode([citation])[0]
    
    # Calculate cosine similarity (1 - cosine distance)
    similarity = 1 - cosine(highlight_embedding, citation_embedding)
    return similarity

def calculate_rouge_score(highlight, citation):
    """Calculate ROUGE-L F-measure score."""
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    scores = scorer.score(highlight, citation)
    return scores['rougeL'].fmeasure

def calculate_tfidf_score(highlight, citation):
    """Calculate TF-IDF similarity score."""
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([highlight, citation])
    
    # Convert sparse matrix to dense array for cosine similarity calculation
    dense_matrix = tfidf_matrix.toarray()
    
    # Calculate cosine similarity
    similarity = 1 - cosine(dense_matrix[0], dense_matrix[1])
    return similarity

def process_data(data):
    """Process the data and reformat to show scores by citation type in columns."""
    results = []
    
    for item in data:
        highlight = item['highlight']
        result_entry = {'id': item['id'], 'highlight': highlight}
        
        # Process each citation type
        for citation_type in ['base_citation', 'ba_citation', 'fo_citation']:
            prefix = citation_type.split('_')[0]  # Extract 'base', 'ba', or 'fo'
            
            if citation_type in item and item[citation_type]:
                citation = item[citation_type]
                
                # Calculate scores
                emb_similarity = calculate_embedding_similarity(highlight, citation)
                rouge_score = calculate_rouge_score(highlight, citation)
                tfidf_score = calculate_tfidf_score(highlight, citation)
                
                # Add scores as columns with prefix
                result_entry[f'{prefix}_emb_similarity'] = emb_similarity
                result_entry[f'{prefix}_rouge_score'] = rouge_score
                result_entry[f'{prefix}_tfidf_score'] = tfidf_score
                
            else:
                # Set default values if citation doesn't exist
                result_entry[f'{prefix}_emb_similarity'] = None
                result_entry[f'{prefix}_rouge_score'] = None
                result_entry[f'{prefix}_tfidf_score'] = None
                
        results.append(result_entry)
    
    return results

# Process the data
results = process_data(linear_results)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/jet/home/iwiryadi/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/opt/packages/AI/pytorch_23.02-1.13.1-py3/lib/python3.10/site-packages/scipy/spatial/distance.py:622: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [25]:
r = pd.DataFrame(results) 
r.drop(['id', 'highlight'], axis=1).mean()

base_emb_similarity   0.4787
base_rouge_score      0.2131
base_tfidf_score      0.1847
ba_emb_similarity     0.6686
ba_rouge_score        0.3411
ba_tfidf_score        0.2947
fo_emb_similarity     0.6661
fo_rouge_score        0.3414
fo_tfidf_score        0.2987
dtype: float64

In [ ]:
# Notes in case jupyter results got deleted:
# 1. linear_results, 500 samples, 9 min 40 s -> at 1.5s / iter, 300k samples needs 125 hours.
# 2. process_data, 500 samples, 5 min 30s    